<a href="https://colab.research.google.com/github/Malaveekasridhar20/Emotional-Campanion-AI/blob/main/final_emotional_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🚀 Install Dependencies
!pip install transformers gradio accelerate sentencepiece -q

# ✅ Import Libraries
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM, pipeline
import gradio as gr

# === Emotion Detection ===
emotion_model_id = "monologg/bert-base-cased-goemotions-original"
emotion_tokenizer = AutoTokenizer.from_pretrained(emotion_model_id)
emotion_model = AutoModelForSequenceClassification.from_pretrained(emotion_model_id)
emotion_labels = emotion_model.config.id2label

def detect_emotion(text):
    inputs = emotion_tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        logits = emotion_model(**inputs).logits
    probs = F.softmax(logits, dim=-1)
    top_prob, top_idx = torch.max(probs, dim=1)
    emotion = emotion_labels[top_idx.item()]
    return emotion, top_prob.item()

# === Emotionally Intelligent LLM (OpenChat 3.5) ===
chat_model_id = "openchat/openchat-3.5-0106"
chat_tokenizer = AutoTokenizer.from_pretrained(chat_model_id)
chat_model = AutoModelForCausalLM.from_pretrained(
    chat_model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# === Emotionally Conditioned Reply Function ===
def generate_response(user_input, emotion):
    system_prompt = f"""<|system|>
You are an emotionally intelligent AI companion.
The user is feeling {emotion.upper()}.
Provide a warm, helpful, and supportive reply.
</s>
<|user|>
{user_input}
<|assistant|>"""

    inputs = chat_tokenizer(system_prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = chat_model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            do_sample=True,
            top_p=0.9
        )
    response = chat_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("<|assistant|>")[-1].strip()

# === Gradio Chat UI ===
def emotional_chat(user_message, history=[]):
    emotion, prob = detect_emotion(user_message)
    reply = generate_response(user_message, emotion)
    history.append((f"[{emotion.upper()}] {user_message}", reply))
    return history, history

# === Interface ===
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Emotional Companion (OpenChat 3.5)\nResponds with empathy based on your feelings.")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Tell me how you're feeling or what happened...")
    state = gr.State([])

    def on_submit(message, state):
        return emotional_chat(message, state)

    msg.submit(on_submit, [msg, state], [chatbot, state])

demo.launch()
